# Miniproyecto #3

Modelos y Simulación - Sección 10

Fecha: 28-09-2020

## Integrantes:
* Oscar Juárez - 17315
* José Cifuentes - 17509

## Importación de librerías


In [1]:
import random
import math
from pprint import pprint

## Funciones útiles

In [2]:
def GenerarNuevoTiempo(instante, LambdaPoisson):
    return instante + exponencial(LambdaPoisson)

def exponencial(Lambda):
    U = random.random()
    return -(1/Lambda)*math.log(U)

def TiempoEntreArreglos(llegadas, salidas):
    suma = 0
    for i in range(len(llegadas)):
        suma += salidas[i]-llegadas[i]
    return suma

def TiempoServidorIdle(llegadas, salidas):
    suma = 0
    for i in range(len(llegadas)-1):
        suma += llegadas[i+1]-salidas[i]
    return suma + (0-llegadas[0])

def ServersAreIdle(SS):
    return True if sum(SS[1:]) == 0 else False

def GetNextIdle(SS):
    SS = SS[1:]
    try:
        return SS.index(0)+1
    except:
        return -1

def GetMinT(serverList):
    reqs = [i['t'] for i in serverList]
    minT = min(reqs)
    return minT, reqs.index(minT)


## Task 1

### Gorilla Megacomputing

Algoritmo para un solo servidor


In [3]:
def simularServer(LambdaExponencial,LambdaPoisson,T):
  # DEFINICIÓN DE VARIABLES 
  llegadas = []
  salidas = []

  llegadasCola = []
  salidasCola = []
  cola=[]

  #Lambda = num. eventos/ num. unidades
  #LambdaPoisson = 2400 / 60
  #LambdaExponencial = 100
  t, Na, Nd, n = 0, 0, 0, 0
  #T = 10

  T0 = GenerarNuevoTiempo(0, LambdaPoisson)
  ta = T0
  td = math.inf

  # Inicio del ciclo while
  while True:
    cola.append(n)  
    # CASO 1
    if ta <= td and ta <= T:
      t = ta
      Na += 1
      n += 1
      
      ta = GenerarNuevoTiempo(t, LambdaPoisson)
      
      if n == 1:
        Y = exponencial(LambdaExponencial)
        Ytemp=Y
        td = t + Y
      else:
        #sabemos que va para cola
        llegadasCola.append(t)
      llegadas.append(t)
    
    # CASO 2
    elif td < ta and td <= T:
      t = td
      n -= 1
      Nd += 1
      
      if n == 0:
        td = math.inf
      else:
        Y = exponencial(LambdaExponencial)
        td = t + Y
        salidasCola.append(t)
      
      salidas.append(t)
          
    # CASO 3
    elif min(ta, td) > T and n > 0:
      t = td
      n -= 1
      Nd += 1
      
      if n > 0:
        Y = exponencial(LambdaExponencial)
        td = t + Y
        salidasCola.append(t)
      salidas.append(t)

    # CASO 4
    elif min(ta, td) > T and n == 0:
      Tp = max(t - T, 0)
      return llegadas, salidas, llegadasCola, salidasCola, cola, Tp
      break

In [4]:
T=3600
llegadas, salidas, llegadasCola, salidasCola, cola,Tp=simularServer(LambdaExponencial=100,LambdaPoisson=2400 / 60,T=3600)
#print(cola)
#print(llegadasCola)
#print(salidasCola)

#### Impresión de salidas


In [5]:
# Print de resultados finales
print('LOS RESULTADOS FINALES SON:\n\n')
print(f'LARGO LLEGADAS: {len(llegadas)} \nLARGO SALIDAS: {len(salidas)} \nLargo de cola: {len(cola)}')

LOS RESULTADOS FINALES SON:


LARGO LLEGADAS: 143757 
LARGO SALIDAS: 143757 
Largo de cola: 287515


#### Resultados proveedor 1

1. ¿Cuántas solicitudes atendió cada servidor?

In [6]:
# Se imprime el largo del arreglo de solicitudes que atendió el servidor
print(f'{len(llegadas)} solicitudes en total')

143757 solicitudes en total


2. ¿Cuánto tiempo estuvo el servidor ocupado?

In [7]:
# Obtenemos una sumatoria de las diferencias entre cada instante que el servidor
# atendió y terminó una solicitud.
print(f'{TiempoEntreArreglos(llegadas, salidas)} segundos')

2391.0953281248335 segundos


3. ¿Cuánto tiempo estuvo el servidor desocupado (idle)?

In [8]:
# Hacemos una sumatoria de la diferencia entre instantes cuando el servidor
# terminó una solicitud y entró otra.
print(f'{TiempoServidorIdle(llegadas, salidas)} segundos')

1208.8407218974305 segundos


4. ¿Cuánto	tiempo	en	total	estuvieron	las	solicitudes en	cola?

In [9]:
tiempoTotalCola = TiempoEntreArreglos(llegadasCola, salidasCola)
print(f'{tiempoTotalCola} segundos')

950.4012304701087 segundos


5. En promedio ¿cuánto tiempo estuvo cada solicitud en cola?

In [10]:
promedioCola = tiempoTotalCola/len(llegadas)
print(f'{promedioCola} segundos')

0.006611164885675888 segundos


6. En promedio, ¿cuántas solicitudes estuvieron en cola cada segundo?

In [11]:
llegadasPorSegundo={}

for i in range(int(max(llegadasCola))+1):
  llegadasPorSegundo[i]=0

for i in llegadasCola:
  llegadasPorSegundo[int(i)]+=1

suma=sum(llegadasPorSegundo.values())
cantTotal=len(llegadasPorSegundo.values())

print(f'En promedio, {int(suma/cantTotal)} solicitudes en cola cada segundo')

En promedio, 15 solicitudes en cola cada segundo


7. ¿Cuál es el momento de la salida de la última solicitud?

In [12]:
print(T+Tp)

3600.0238133982443


### Ant Smart Computing

#### Algoritmo para servidores dinámicos

In [13]:
def simularServerParalelo(cantServers,LambdaExponencial,LambdaPoisson,T):
  # ------------ Inicializacion ------------
  #Lambda = num. eventos/ num. unidades

  # CAMBIAR CANT. DE SERVERS
  #cantServers = 4

  servers = [{
      'llegadas': {},
      'salidas': {},
      't': math.inf
  } for i in range(cantServers)]

  #LambdaPoisson = 2400 / 60
  #LambdaExponencial = 100
  t, Na,Nd = 0,0,0
  SS = [0 for i in range(cantServers + 1)]
  tA = GenerarNuevoTiempo(0, LambdaPoisson)
  #T = 0.1

  cola = {}

  while True:
      # CASO 1

      # Obtenemos el t más pequeño de entre todos los serivdores
      minT, indexMinT = GetMinT(servers)

      # nuevo ingreso en tiempo tA
      if (tA == min(tA, minT) and tA < T):
          # print('CASO1')
          t = tA
          Na += 1
          tA = GenerarNuevoTiempo(t, LambdaPoisson)
          # Obtenemos el próximo servidor que está idle (0)
          idleServer = GetNextIdle(SS)

          if (SS[0] == 0 and ServersAreIdle(SS)):

              SS[1] = Na
              SS[0] = 1

              # todo libre
              servers[0]['llegadas'][Na] = t
              servers[0]['t'] = t + exponencial(LambdaExponencial)

          elif (SS[0] > 0 and idleServer != -1):
              # un cliente y servidor 2 libre
              SS[idleServer] = Na
              SS[0] = SS[0]+1

              # En el servidor correspondiente, marcamos el instante de llegada y el t
              servers[idleServer-1]['llegadas'][Na] = t
              servers[idleServer-1]['t'] = t + exponencial(LambdaExponencial)

          elif SS[0] > len(servers)-1:
              cola[Na] = [t, None]
              SS[0] = SS[0] + 1

          continue

      elif minT < tA:
          t = servers[indexMinT]['t']

          # Obtenemos el id de la solicitud terminada
          # Se suma 1 a indexMintT porque estamos en SS (en donde la primera posición es
          # la cantidad de solicitudes en el sistema)
          solicitudActual = SS[indexMinT + 1]

          # Actualizamos la salida de la solicitud correspondiente
          servers[indexMinT]['salidas'][solicitudActual] = t

          if 0 < SS[0] <= cantServers:
              SS[0] = SS[0]-1
              SS[indexMinT+1] = 0
              servers[indexMinT]['t'] = math.inf

          elif SS[0] > cantServers:
              sigSolicitud = max(SS[1:]) + 1

              SS[indexMinT+1] = sigSolicitud
              SS[0] = SS[0] - 1

              cola[sigSolicitud][1] = indexMinT+1
              servers[indexMinT]['llegadas'][sigSolicitud] = t

              servers[indexMinT]['t'] = t + exponencial(LambdaExponencial)

          continue

      elif SS[0] > 0:

        t = servers[indexMinT]['t']
        solicitudActual = SS[indexMinT + 1]

        # Actualizamos la salida de la solicitud correspondiente
        servers[indexMinT]['salidas'][solicitudActual] = t + \
          exponencial(LambdaExponencial)
        servers[indexMinT]['t'] = math.inf
        # Actualizamos cantidad de solicitudes en el sistema
        SS[0] = SS[0] - 1

        continue


      # pprint(servers)
      # print('\nCOLA:')
      # pprint(cola)
      #print('Tiempo extra '+str(max(t-T, 0)))
      #print('\nUltimo instante '+str(t))
      #print('SS='+str(SS))
      return servers, cola, t
      break


In [14]:
registroServers, registroCola, ultimoInstante=simularServerParalelo(cantServers=10,LambdaExponencial=10,LambdaPoisson=2400 / 60,T=3600)

#### Resultados proveedor 2

1. ¿Cuántas solicitudes atendió cada servidor?

In [15]:
# Se imprime el largo del arreglo de solicitudes que atendió el servidor
for i in range(len(registroServers)):
  total=len(registroServers[i]['llegadas'])
  print(f'Servidor {i} atendio un total de {total} solicitudes')

Servidor 0 atendio un total de 29042 solicitudes
Servidor 1 atendio un total de 26762 solicitudes
Servidor 2 atendio un total de 23683 solicitudes
Servidor 3 atendio un total de 20259 solicitudes
Servidor 4 atendio un total de 16047 solicitudes
Servidor 5 atendio un total de 11899 solicitudes
Servidor 6 atendio un total de 7924 solicitudes
Servidor 7 atendio un total de 4728 solicitudes
Servidor 8 atendio un total de 2505 solicitudes
Servidor 9 atendio un total de 1260 solicitudes


2. ¿Cuánto tiempo estuvo el servidor ocupado?

In [16]:
# Obtenemos una sumatoria de las diferencias entre cada instante que el servidor
# atendió y terminó una solicitud.

for i in range(len(registroServers)):
  try:
    llegadasDict=registroServers[i]['llegadas']
    salidasDict=registroServers[i]['salidas']
    llegadasList=[]
    salidasList=[]
    for id in registroServers[0]['llegadas'].keys():
      llegadasList.append(llegadasDict[id])
      salidasList.append(salidasDict[id])
    
    print(f'Servidor {i} estuvo ocupado {TiempoEntreArreglos(llegadasList, salidasList)} segundos')
  except:
    print(f'Servidor {i} estuvo ocupado 0 segundos')
    
#print(f'{TiempoEntreArreglos(llegadas, salidas)} segundos')

Servidor 0 estuvo ocupado 2877.7804120272276 segundos
Servidor 1 estuvo ocupado 0 segundos
Servidor 2 estuvo ocupado 0 segundos
Servidor 3 estuvo ocupado 0 segundos
Servidor 4 estuvo ocupado 0 segundos
Servidor 5 estuvo ocupado 0 segundos
Servidor 6 estuvo ocupado 0 segundos
Servidor 7 estuvo ocupado 0 segundos
Servidor 8 estuvo ocupado 0 segundos
Servidor 9 estuvo ocupado 0 segundos


3. ¿Cuánto tiempo estuvo el servidor desocupado (idle)?

In [17]:
# Hacemos una sumatoria de la diferencia entre instantes cuando el servidor
# terminó una solicitud y entró otra.
for i in range(len(registroServers)):
  try:
    llegadasDict=registroServers[i]['llegadas']
    salidasDict=registroServers[i]['salidas']
    llegadasList=[]
    salidasList=[]
    for id in registroServers[0]['llegadas'].keys():
      llegadasList.append(llegadasDict[id])
      salidasList.append(salidasDict[id])
    
    print(f'Servidor {i} estuvo desocupado {TiempoServidorIdle(llegadasList, salidasList)} segundos')
  except:
    print(f'Servidor {i} estuvo desocupado todo el tiempo')
#print(f'{TiempoServidorIdle(llegadas, salidas)} segundos')

Servidor 0 estuvo desocupado 722.2515803523368 segundos
Servidor 1 estuvo desocupado todo el tiempo
Servidor 2 estuvo desocupado todo el tiempo
Servidor 3 estuvo desocupado todo el tiempo
Servidor 4 estuvo desocupado todo el tiempo
Servidor 5 estuvo desocupado todo el tiempo
Servidor 6 estuvo desocupado todo el tiempo
Servidor 7 estuvo desocupado todo el tiempo
Servidor 8 estuvo desocupado todo el tiempo
Servidor 9 estuvo desocupado todo el tiempo


4. ¿Cuánto	tiempo	en	total	estuvieron	las	solicitudes en	cola?

In [18]:
if (len(registroCola)>0):
  llegadasCola=[]
  salidasCola=[]
  for i in registroCola.keys():
    llegadasCola.append(registroCola[i][0])
    idServidor=registroCola[i][1]
    idProceso=i
    salidasCola.append(registroServers[idServidor-1]['llegadas'][idProceso])
  tiempoTotalCola = TiempoEntreArreglos(llegadasCola, salidasCola)
  print(f'{tiempoTotalCola} segundos')
else:
  print('No hay cola')


20.19867348360672 segundos


5. En promedio ¿cuánto tiempo estuvo cada solicitud en cola?

In [19]:
idSolicitudes=[]
for i in registroServers:
  idSolicitudes+=i['llegadas'].keys()
promedioCola = tiempoTotalCola/max(idSolicitudes)
print(f'{promedioCola} segundos')

0.00014016247065489816 segundos


6. En promedio, ¿cuántas solicitudes estuvieron en cola cada segundo?

In [20]:
instantesDeIngresoCola=[]
for i in registroCola.values():
  instantesDeIngresoCola.append(i[0])


llegadasPorSegundo={}

for i in range(int(max(instantesDeIngresoCola))+1):
  llegadasPorSegundo[i]=0



for i in instantesDeIngresoCola:
  llegadasPorSegundo[int(i)]+=1


suma=sum(llegadasPorSegundo.values())
cantTotal=len(llegadasPorSegundo.values())

print(f'En promedio, {int(suma/cantTotal)} solicitudes en cola cada segundo')


En promedio, 0 solicitudes en cola cada segundo


7. ¿Cuál es el momento de la salida de la última solicitud?

In [21]:
print(ultimoInstante)

3600.177624604643


## Task 2

In [22]:
print(len(registroCola))
cantidadServers=10
while True:
  _, registroCola, _=simularServerParalelo(cantServers=cantidadServers,LambdaExponencial=10,LambdaPoisson=2400 / 60,T=3600)
  if (len(registroCola)==0):
    break
  cantidadServers+=1
print(cantidadServers)

1229
16


## Task 3 y 4

Repetimos el procedimiento anterior, pero con valores distintos y evaluamos las preguntas.

In [23]:
LambdaPoisson = 6000/60
T = 3600

### Gorilla Megacomputing

Simulamos la corrida con el nuevo valor de lambda.

In [24]:
llegadas, salidas, llegadasCola, salidasCola, cola,Tp = simularServer(
    LambdaExponencial=100,
    LambdaPoisson=LambdaPoisson,
    T=T
  )

print(f'LARGO LLEGADAS: {len(llegadas)} \nLARGO SALIDAS: {len(salidas)} \nLargo de cola: {len(cola)}')

LARGO LLEGADAS: 360182 
LARGO SALIDAS: 360182 
Largo de cola: 720365


#### Análisis de resultados

1. ¿Cuántas solicitudes atendió cada servidor?

In [25]:
# Se imprime el largo del arreglo de solicitudes que atendió el servidor
print(f'{len(llegadas)} solicitudes en total')

360182 solicitudes en total


2. ¿Cuánto tiempo estuvo el servidor ocupado?

In [26]:
# Obtenemos una sumatoria de las diferencias entre cada instante que el servidor
# atendió y terminó una solicitud.
segundos = int(TiempoEntreArreglos(llegadas, salidas))
minutos = int(segundos/60)
horas = int(minutos/60)
print(f'{horas} horas == {minutos} minutos == {segundos} segundos')

425 horas == 25535 minutos == 1532140 segundos


3. ¿Cuánto tiempo estuvo el servidor desocupado (idle)?

In [27]:
# Hacemos una sumatoria de la diferencia entre instantes cuando el servidor
# terminó una solicitud y entró otra.
segundos = TiempoServidorIdle(llegadas, salidas) 
minutos = int(segundos/60)
horas = int(minutos/60)
print(f'{horas} horas == {minutos} minutos == {segundos} segundos')

-424 horas == -25475 minutos == -1528528.735150737 segundos


4. ¿Cuánto	tiempo	en	total	estuvieron	las	solicitudes en	cola?

In [28]:
tiempoTotalCola = TiempoEntreArreglos(llegadasCola, salidasCola)
minutos = int(tiempoTotalCola/60)
horas = int(minutos/60)
print(f'{horas} horas == {minutos} minutos == {tiempoTotalCola} segundos')

424 horas == 25475 minutos == 1528533.316503449 segundos


5. En promedio ¿cuánto tiempo estuvo cada solicitud en cola?

In [29]:
promedioCola = tiempoTotalCola/len(llegadas)
print(f'{promedioCola} segundos')

4.243780412412194 segundos


6. En promedio, ¿cuántas solicitudes estuvieron en cola cada segundo?

In [30]:
llegadasPorSegundo={}

for i in range(int(max(llegadasCola))+1):
  llegadasPorSegundo[i]=0

for i in llegadasCola:
  llegadasPorSegundo[int(i)]+=1

suma=sum(llegadasPorSegundo.values())
cantTotal=len(llegadasPorSegundo.values())

print(f'En promedio, {int(suma/cantTotal)} solicitudes en cola cada segundo')

En promedio, 99 solicitudes en cola cada segundo


7. ¿Cuál es el momento de la salida de la última solicitud?

In [31]:
print(f'En el segundo {T+Tp} == minuto {(T+Tp)/60}')

En el segundo 3611.2871031774534 == minuto 60.18811838629089


### Ant Smart Computing

Nuevamente, realizamos otra simulación con los nuevos valores

In [32]:
registroServers, registroCola, ultimoInstante = simularServerParalelo(
    cantServers=10,
    LambdaExponencial=10,
    LambdaPoisson=LambdaPoisson,
    T=T
  )

#### Análisis de resultados

1. ¿Cuántas solicitudes atendió cada servidor?

In [33]:
# Se imprime el largo del arreglo de solicitudes que atendió el servidor
for i in range(len(registroServers)):
  total=len(registroServers[i]['llegadas'])
  print(f'Servidor {i} atendio un total de {total} solicitudes')

Servidor 0 atendio un total de 35905 solicitudes
Servidor 1 atendio un total de 35951 solicitudes
Servidor 2 atendio un total de 35646 solicitudes
Servidor 3 atendio un total de 35681 solicitudes
Servidor 4 atendio un total de 35867 solicitudes
Servidor 5 atendio un total de 35857 solicitudes
Servidor 6 atendio un total de 36162 solicitudes
Servidor 7 atendio un total de 35907 solicitudes
Servidor 8 atendio un total de 35906 solicitudes
Servidor 9 atendio un total de 35990 solicitudes


2. ¿Cuánto tiempo estuvo el servidor ocupado?

In [34]:
# Obtenemos una sumatoria de las diferencias entre cada instante que el servidor
# atendió y terminó una solicitud.

for i in range(len(registroServers)):
  try:
    llegadasDict=registroServers[i]['llegadas']
    salidasDict=registroServers[i]['salidas']
    llegadasList=[]
    salidasList=[]
    for id in registroServers[0]['llegadas'].keys():
      llegadasList.append(llegadasDict[id])
      salidasList.append(salidasDict[id])
    
    print(f'Servidor {i} estuvo ocupado {TiempoEntreArreglos(llegadasList, salidasList)} segundos')
  except:
    print(f'Servidor {i} estuvo ocupado 0 segundos')
    
#print(f'{TiempoEntreArreglos(llegadas, salidas)} segundos')

Servidor 0 estuvo ocupado inf segundos
Servidor 1 estuvo ocupado 0 segundos
Servidor 2 estuvo ocupado 0 segundos
Servidor 3 estuvo ocupado 0 segundos
Servidor 4 estuvo ocupado 0 segundos
Servidor 5 estuvo ocupado 0 segundos
Servidor 6 estuvo ocupado 0 segundos
Servidor 7 estuvo ocupado 0 segundos
Servidor 8 estuvo ocupado 0 segundos
Servidor 9 estuvo ocupado 0 segundos


3. ¿Cuánto tiempo estuvo el servidor desocupado (idle)?

In [35]:
# Hacemos una sumatoria de la diferencia entre instantes cuando el servidor
# terminó una solicitud y entró otra.
for i in range(len(registroServers)):
  try:
    llegadasDict=registroServers[i]['llegadas']
    salidasDict=registroServers[i]['salidas']
    llegadasList=[]
    salidasList=[]
    for id in registroServers[0]['llegadas'].keys():
      llegadasList.append(llegadasDict[id])
      salidasList.append(salidasDict[id])
    
    print(f'Servidor {i} estuvo desocupado {TiempoServidorIdle(llegadasList, salidasList)} segundos')
  except:
    print(f'Servidor {i} estuvo desocupado todo el tiempo')


Servidor 0 estuvo desocupado 0.25745831535005104 segundos
Servidor 1 estuvo desocupado todo el tiempo
Servidor 2 estuvo desocupado todo el tiempo
Servidor 3 estuvo desocupado todo el tiempo
Servidor 4 estuvo desocupado todo el tiempo
Servidor 5 estuvo desocupado todo el tiempo
Servidor 6 estuvo desocupado todo el tiempo
Servidor 7 estuvo desocupado todo el tiempo
Servidor 8 estuvo desocupado todo el tiempo
Servidor 9 estuvo desocupado todo el tiempo


4. ¿Cuánto	tiempo	en	total	estuvieron	las	solicitudes en	cola?

In [36]:
if (len(registroCola)>0):
  llegadasCola=[]
  salidasCola=[]
  for i in registroCola.keys():
    llegadasCola.append(registroCola[i][0])
    idServidor=registroCola[i][1]
    idProceso=i
    salidasCola.append(registroServers[idServidor-1]['llegadas'][idProceso])
  tiempoTotalCola = TiempoEntreArreglos(llegadasCola, salidasCola)
  print(f'{tiempoTotalCola} segundos')
else:
  print('No hay cola')


TypeError: ignored

5. En promedio ¿cuánto tiempo estuvo cada solicitud en cola?

In [37]:
idSolicitudes=[]
for i in registroServers:
  idSolicitudes+=i['llegadas'].keys()
promedioCola = tiempoTotalCola/max(idSolicitudes)
print(f'{promedioCola} segundos')

4.259271596846366 segundos


6. En promedio, ¿cuántas solicitudes estuvieron en cola cada segundo?

In [38]:
instantesDeIngresoCola=[]
for i in registroCola.values():
  instantesDeIngresoCola.append(i[0])


llegadasPorSegundo={}

for i in range(int(max(instantesDeIngresoCola))+1):
  llegadasPorSegundo[i]=0



for i in instantesDeIngresoCola:
  llegadasPorSegundo[int(i)]+=1


suma=sum(llegadasPorSegundo.values())
cantTotal=len(llegadasPorSegundo.values())

print(f'En promedio, {int(suma/cantTotal)} solicitudes en cola cada segundo')


En promedio, 99 solicitudes en cola cada segundo


7. ¿Cuál es el momento de la salida de la última solicitud?

In [39]:
print(ultimoInstante)

inf


## Task 5

#### Recomendación a junta directiva:

Aspectos a tomar en cuenta:

* Solicitudes totales
* Tiempo del sistema Ocupado e IDLE
* Tiempo total y promedio de solicitudes en cola
* Promedio de solicitudes en cola por segundo